# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

Graph databases are a natural way of navigating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metadata on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions with python syntax. Submit the results by following the instructions.


### Questions

1. How many distinct countries exist in the database? [Hint: mind repetitions]
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [NB: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering the number of users who reviewed it? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews of beers, taking into consideration the "overall" score they assign as a weight, how many communities are formed from these relationships? How many users are in the largest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations, or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explanations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself (please submit one Jupyter notebook per group).

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided on Moodle (apr. 6 Gb).
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Homework\Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided on Moodle (it is the same plugins as for the labs).
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins`.
- Change the code below to include your correct folder paths. Be sure that you have Docker running first:

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins":/plugins -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\HW1\Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted="gds.*" --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit  before you can view the data on Neo4j webpage (let's say 3 minutes).

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**XXXXXX**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|Burcu Yesilyurt|20230763|
|David Gustavo Guarin Rojas|20230602|
|Ilyass Jannah|20230598|


#### 1. How many distinct countries exist in the database?

In [28]:
query = """
        MATCH (c:COUNTRIES) 
        RETURN count(DISTINCT c.name) AS countries
    """

result = execute_read(driver, query)

pprint(result)

[<Record countries=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [30]:
# 2.A) Get the number of relationships between beers and reviews then order the results by the number of reviews
query = """
      MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
      RETURN b.name AS beer_name, count(r) AS number_of_reviews
      ORDER BY number_of_reviews DESC
      LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='IPA' number_of_reviews=31387>]


In [31]:
#2.B) Get the number of relationships between breweries and reviews then order the results by the number of reviews
query = """
        MATCH(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN br.name AS brewery_name, COUNT(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1
        """

result = execute_read(driver, query)

pprint(result)

[<Record brewery_name='Sierra Nevada Brewing Co.' review_count=175161>]


In [32]:
#2.C) Get the number of relationships between countries and reviews passing by other nodes then order the results by the 
#number of reviews
query = """
        MATCH (c:COUNTRIES)-[]-(:CITIES)-[:IN]-(:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN c.name AS countries_index, COUNT(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1
        """

result = execute_read(driver, query)

pprint(result)

[<Record countries_index='US' review_count=7675804>]


#### 3. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [33]:
query = """
    MATCH (u:USER)-[:POSTED]-(r:REVIEWS)-[:REVIEWED]-(b:BEERS)-[]-()-[]-(:USER{name:'CTJman'})
    RETURN u.name AS username, count(r) AS review_count
    ORDER BY review_count DESC
    LIMIT 5
"""
result = execute_read(driver, query)

pprint(result)

[<Record username='acurtis' review_count=1428>,
 <Record username='Texasfan549' review_count=1257>,
 <Record username='kjkinsey' review_count=1205>,
 <Record username='oline73' review_count=1191>,
 <Record username='chippo33' review_count=1161>]


#### 4. Which Portuguese brewery has the most beers?


In [34]:
query = """
       MATCH (:COUNTRIES{name:'PT'})-[]-(:CITIES)-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)
       RETURN br.name AS brewery_name, COUNT(b) AS beer_number
       ORDER BY beer_number DESC
       LIMIT 1;
        """

result = execute_read(driver, query)

pprint(result)

[<Record brewery_name='Dois Corvos Cervejeira' beer_number=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [35]:
query = """
       MATCH (:BREWERIES{name:'Dois Corvos Cervejeira'})-[:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
       RETURN b.name AS beer_name, COUNT(r) AS review_count
       ORDER BY review_count DESC
       LIMIT 1;
        """

result = execute_read(driver, query)

pprint(result)

[<Record beer_name='Finisterra' review_count=10>]


#### 6. On average how many different beer styles does each brewery produce?

In [36]:
query = """
      MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
      WITH br, COUNT(DISTINCT s) AS stylesNum
      RETURN AVG(stylesNum) AS average;
       """

result = execute_read(driver, query)

pprint(result)

[<Record average=10.595992516144538>]


#### 7. Which brewery produces the strongest beers according to ABV?

In [38]:
query = """
       // Relationship between the breweries and their beers
       MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
       // Condition so that the abv is an actual number so we can compare
       WHERE b.abv IS NOT NULL AND b.abv <> 'nan'
       RETURN br.name AS brewery_name, b.abv AS beer_abv
       ORDER BY beer_abv DESC
       LIMIT 1;
       """

result = execute_read(driver, query)

pprint(result)

[<Record brewery_name='New Belgium Brewing Company' beer_abv='9.99'>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [39]:
# This cell give a reccomendation based on the beer with the best smell and look depending on the reviews
query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        // Making sure that look and smell are valid number to compare(not null and nan)
        WHERE r.look IS NOT NULL AND r.look <> 'nan' AND r.smell IS NOT NULL AND r.smell <> 'nan'
        WITH b, max(r.look) AS max_look, max(r.smell) AS max_smell
        MATCH (b2:BEERS)-[:HAS_STYLE]->(s:STYLE)
        // Get the style of the beer with max smell and look
        WHERE b2.name = b.name
        RETURN s.name AS style_name, b.name AS beer_name, max_look, max_smell
        ORDER BY max_look DESC, max_smell DESC
        LIMIT 1;
       """

result = execute_read(driver, query)

pprint(result)

[<Record style_name='Russian Imperial Stout' beer_name='Konrads Stout Russian Imperial Stout' max_look='5.0' max_smell='5.0'>]


In [40]:
# Calculate the avg of smell and look of the beers and recommend beers that have the best avg in both categories
query ="""
        MATCH (s:STYLE)<-[:HAS_STYLE]-(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        // Making sure smell and look have valid types
        WHERE r.look IS NOT NULL AND r.look <> 'nan' AND r.smell IS NOT NULL AND r.smell <> 'nan'
        // Change smell and look to type float because we need to aggregate by avg and can't do that on strings
        WITH s.name as name, toFloat(r.smell) as smell, toFloat(r.look) as look
        RETURN name, AVG(smell) as AvgSmell, AVG(look) as AvgLook
        ORDER BY AvgSmell DESC, AvgLook DESC
        LIMIT 3
"""

result = execute_read(driver, query)
pprint(result)

[<Record name='New England IPA' AvgSmell=4.413614764761178 AvgLook=4.383595613210877>,
 <Record name='American Imperial Stout' AvgSmell=4.262885690516247 AvgLook=4.286394107776094>,
 <Record name='Belgian Gueuze' AvgSmell=4.230876612146052 AvgLook=4.17496417453178>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    A. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    B. Which beer is the most influential when considering the number of users who reviewed it?
    C. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users are in the largest community?

We chose to answer question B. Which beer is the most influential when considering the number of users who reviewed it?

In [18]:
try:
    query = """
            CALL gds.graph.drop('hw_9b') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='hw_9b'>]


In [19]:
#We have limit 10000 because if more we dont have the processor to it.
#Create the graph here the weight is not something we will consider more the sourcve and target.
try: 
    query_projection = """
        CALL gds.graph.project.cypher(
            'hw_9b',
            'MATCH (n:BEERS) RETURN id(n) AS id',
            'MATCH (b1:BEERS)-[:REVIEWED]-(r:REVIEWS)-[:POSTED]-(u:USER)<-[:POSTED]-(r2:REVIEWS)<-[:REVIEWED]-(b2:BEERS)
             WITH r.overall as scores, b1, b2, u
             RETURN id(b1) AS source, id(b2) AS target, scores AS weight
             limit 10000
             ' 
        )
    """
  
    result = execute_read(driver, query_projection)
    
    pprint(result)
    print('\n')
except Exception as e:
  
    pprint(e)

[<Record nodeQuery='MATCH (n:BEERS) RETURN id(n) AS id' relationshipQuery='MATCH (b1:BEERS)-[:REVIEWED]-(r:REVIEWS)-[:POSTED]-(u:USER)<-[:POSTED]-(r2:REVIEWS)<-[:REVIEWED]-(b2:BEERS)\n             WITH r.overall as scores, b1, b2, u\n             RETURN id(b1) AS source, id(b2) AS target, scores AS weight\n             limit 10000' graphName='hw_9b' nodeCount=717746 relationshipCount=10000 projectMillis=1418>]




In [20]:
#Using pagerank to check on how it influences.
try:
    query = """
        CALL gds.pageRank.stream('hw_9b')
            
            YIELD nodeId, score
            
            RETURN gds.util.asNode(nodeId).name AS name, score
            
            ORDER BY score desc
            LIMIT 7
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record name='Verboten' score=612.6015453441576>,
 <Record name='I Hardcore You' score=612.6015453441576>,
 <Record name='Red Legacy' score=612.6015453441576>,
 <Record name='Arcadia Bourbon Barrel Aged Cereal Killer Barleywine' score=553.5210258613263>,
 <Record name='World Burp Beer 2002' score=0.15000000000000002>,
 <Record name='Icon Sender' score=0.15000000000000002>,
 <Record name='Scottish Right' score=0.15000000000000002>]


#### 10. Using Graph Algorithms answer **one** of the following questions:
    A. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    B. Which user is the most influential when it comes to reviews made?

We chose to answer question A. Which beer has the most similar reviews as the beer `Super Bock Stout`?

In [21]:
try:
    query = """
            CALL gds.graph.drop('hw_10a') YIELD graphName;
        """
    
    result = execute_read(driver, query)
except Exception as e:
   
    pprint('Graph did not exist. Creating a new one')

'Graph did not exist. Creating a new one'


In [22]:
# We create first the graph here for which then we use the similarity to get the most similarity
try:
    query_projection = """
 CALL gds.graph.project.cypher(
  'hw_10a',
  'MATCH (b:BEERS) RETURN id(b) AS id',
  'MATCH (targetBeer:BEERS {name: "Super Bock Stout"})-[:REVIEWED]->(:REVIEWS)-[:POSTED]->(reviewers:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b1:BEERS)
   WHERE b1.name <> "Super Bock Stout"
   WITH b1, targetBeer
   RETURN id(targetBeer) AS target, id(b1) AS source'
)
    """

   
    result = execute_read(driver, query_projection)
   
    pprint(result)
    print('\n')
except Exception as e:
   
    pprint(e)

[<Record nodeQuery='MATCH (b:BEERS) RETURN id(b) AS id' relationshipQuery='MATCH (targetBeer:BEERS {name: "Super Bock Stout"})-[:REVIEWED]->(:REVIEWS)-[:POSTED]->(reviewers:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b1:BEERS)\n   WHERE b1.name <> "Super Bock Stout"\n   WITH b1, targetBeer\n   RETURN id(targetBeer) AS target, id(b1) AS source' graphName='hw_10a' nodeCount=717746 relationshipCount=91086 projectMillis=31232>]




In [23]:
try:
    query = """
   CALL gds.nodeSimilarity.stream('hw_10a'  )
   
            YIELD node1, node2, similarity
            WITH gds.util.asNode(node1).name AS beer1, 
            
            gds.util.asNode(node2).name AS beer2, similarity
            
            RETURN beer1, beer2, similarity
            
            ORDER BY similarity ASCENDING
            LIMIT 10
    
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record beer1='Konrads Stout Russian Imperial Stout' beer2='Lomond Gold' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='La Blanche De Drummond' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Vibin’' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Magic #411 - Wild Strawberry/Rhubarb/Vanilla Smoothie IPA' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Three Flowers' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Marina The Grim North Black IPA' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Redhook Double Black Stout' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Fortuna' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2='Rock Ice' similarity=1.0>,
 <Record beer1='Konrads Stout Russian Imperial Stout' beer2="Leinenkugel's Ginger Shandy" similarit

---------------------------------------------------------------------------------------------------------------------------

In [24]:
#This was an idea  that we where getting of trying another aproach with page rank but due to the heap space taking a lot of the
#computer processor we decided on the answer before.
try:
    query = """
            CALL gds.graph.drop('beer_similarity_graph') YIELD graphName;
        """
    
    result = execute_read(driver, query)
except Exception as e:
   
    pprint('Graph did not exist. Creating a new one')

'Graph did not exist. Creating a new one'


In [25]:
try:
    query_projection = """
 CALL gds.graph.project.cypher(
  'beer_similarity_graph',
  'MATCH (b:BEERS) RETURN id(b) AS id',
  'MATCH (b1:BEERS)-[:REVIEWED]->(:REVIEWS)-[:POSTED]->(reviewers:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b2:BEERS)
   WHERE b1.name <> "Super Bock Stout" AND b2.name <> "Super Bock Stout"
   WITH b1, b2, count(DISTINCT reviewers) AS similarityCount
   WHERE rand() < 0.01
   RETURN id(b1) AS source, id(b2) AS target, similarityCount AS similarity
'
)
    """

   
    result = execute_read(driver, query_projection)
   
    pprint(result)
    print('\n')
except Exception as e:
   
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.project.cypher`: Caused by: java.lang.OutOfMemoryError: Java heap space: failed reallocation of scalar replaced objects')


In [26]:
try:
    query = """
       CALL gds.pageRank.stream('beer_similarity_graph')
            YIELD nodeId, score
            WITH gds.util.asNode(nodeId) AS beer, score
            WHERE beer.name <> "Super Bock Stout"
            RETURN beer.name AS similarBeer, score
            ORDER BY score DESC
            LIMIT 5
    
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.pageRank.stream`: Caused by: java.util.NoSuchElementException: Graph with name `beer_similarity_graph` does not exist on database `neo4j`. It might exist on another database.')


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [27]:
try:
    query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
    WHERE r.overall IS NOT NULL AND r.overall <> 'nan' AND r.score IS NOT NULL AND r.score <> 'nan'
    WITH b, AVG(toFloat(r.score)) AS avgScore, count(r) AS reviewCount
    ORDER BY reviewCount DESC
    LIMIT 3
    RETURN b.name AS BeerName, avgScore AS AverageScore, reviewCount AS ReviewCount;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record BeerName='Breakfast Stout' AverageScore=4.473797707349983 ReviewCount=7415>,
 <Record BeerName='KBS (Kentucky Breakfast Stout)' AverageScore=4.560450156205614 ReviewCount=7042>,
 <Record BeerName='Pliny The Elder' AverageScore=4.603536550129994 ReviewCount=6922>]


In [ ]:
#We decided that the best beers are the ones that have the best score, but also the most review count as the
#more the review count the more likely and accurate is the score. 